In [192]:
import pandas as pd
import time
from datetime import datetime as dt

In [ ]:
# 2. Attribution Allocation

## Considering the user converted with revenue generated: 
## and considering spend on that channel worths 
### Payment_period >= 1

In [193]:
df = pd.read_csv('subscribers-updated.csv',parse_dates=['account_creation_date'])
df = df.drop(['Unnamed: 0'], axis=1)
spe = pd.read_csv('channel_spend_graduate.csv')
spe1 = pd.read_csv('channel_spend_graduate.csv')

/Users/spencerzhang/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [194]:
att = df[['subid', 'attribution_technical','attribution_survey','account_creation_date','payment_period','trial_completed']]
att.dropna(subset = ['attribution_survey'],inplace = True)

<ipython-input-194-797dae549caa>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  att.dropna(subset = ['attribution_survey'],inplace = True)


In [195]:
#filtering out paymeent_period == 0 and trial_completed = True
att1 = att[(att['payment_period']!=0) & (att['trial_completed'] == True)]
att1.reset_index(inplace=True)
att1.drop('index',axis='columns', inplace=True)

/Users/spencerzhang/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [100]:
att1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130884 entries, 0 to 130883
Data columns (total 6 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   subid                  130884 non-null  int64         
 1   attribution_technical  130884 non-null  object        
 2   attribution_survey     130884 non-null  object        
 3   account_creation_date  130884 non-null  datetime64[ns]
 4   payment_period         130884 non-null  int64         
 5   trial_completed        130884 non-null  bool          
dtypes: bool(1), datetime64[ns](1), int64(2), object(2)
memory usage: 5.1+ MB


In [196]:
att1.loc[:,('date')] = att1['account_creation_date'].dt.strftime('%Y%m')
att1.loc[:,('date')] = att1['date'].astype('int')
att1

/Users/spencerzhang/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/spencerzhang/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


,subid,attribution_technical,attribution_survey,account_creation_date,payment_period,trial_completed,date
0,25939818,affiliate,referral,2020-03-13 23:54:00,1,True,202003
1,25681720,other,facebook,2020-03-13 23:53:00,1,True,202003
2,22007157,email,tv,2020-03-13 23:51:00,1,True,202003
3,21952559,brand sem intent google,facebook,2020-03-13 23:49:00,1,True,202003
4,22513544,facebook,referral,2020-03-13 23:49:00,1,True,202003
...,...,...,...,...,...,...,...
130879,23371118,facebook,other,2019-06-30 00:17:00,2,True,201906
130880,25780992,facebook,facebook,2019-06-30 00:10:00,3,True,201906
130881,20761569,facebook,facebook,2019-06-30 00:07:00,1,True,201906
130882,27196054,facebook,facebook,2019-06-30 00:01:00,3,True,201906


In [65]:
#for item in timelist:
    #a = att1[att1['date']== item]['attribution_technical'].value_counts()
    #for key in chanlist:
        #refer.at[item, key] = a[key]

### Attribution_Technical 

In [197]:
for m in range(0,len(spe)):
    channel = spe.loc[m,'channel']
    date = spe.loc[m,'date']
    subattri= att1[(att1['date'] == date) & (att1['attribution_technical'] == channel)]
    spe.loc[m,'count'] = len(subattri)
    spe.loc[m,'Average_CAC_Tech'] = spe.loc[m,'spend_AED']/spe.loc[m,'count']

In [114]:
#len(att1[(att1['date'] == 201912)])

In [115]:
#len(att1[(att1['date'] == 201912) & (att1['attribution_technical'] == 'bing')])

In [116]:
#len(att1[(att1['date'] == 201912)])

In [198]:
speMt = spe.sort_values(['channel','spend_AED'],ascending=True)
speMt.reset_index(inplace=True)
speMt.drop('index',axis='columns', inplace=True)

In [199]:
for n in range(1,len(speMt)):
    if speMt.loc[n,'channel'] == speMt.loc[(n-1),'channel']:
        speMt.loc[n,'Marginal_CAC_Tech'] = (speMt.loc[n,'spend_AED'] - speMt.loc[(n-1),'spend_AED'])/(speMt.loc[n,'count'] - speMt.loc[(n-1),'count'])
    else:
        continue

for m in [0,8,16,24,32]:
    speMt.loc[m,'Marginal_CAC_Tech'] = speMt.loc[m,'spend_AED'] /speMt.loc[m,'count']

<ipython-input-199-c0b56dd6f76c>:3: RuntimeWarning: divide by zero encountered in true_divide
  speMt.loc[n,'Marginal_CAC_Tech'] = (speMt.loc[n,'spend_AED'] - speMt.loc[(n-1),'spend_AED'])/(speMt.loc[n,'count'] - speMt.loc[(n-1),'count'])


In [208]:
speMt.to_csv("Attri_Technical.csv")

### Attribution_Survey

In [128]:
att1['attribution_survey'].unique()

array(['referral', 'facebook', 'tv', 'ott', 'pinterest', 'other',
       'podcast', 'search', 'public_radio', 'amfm_radio',
       'social_organic', 'youtube', 'facebook_organic', 'affiliate',
       'sirius xm', 'direct_mail'], dtype=object)

In [201]:
for m in range(0,len(spe1)):
    channel = spe1.loc[m,'channel']
    date = spe1.loc[m,'date']
    subattri= att1[(att1['date'] == date) & (att1['attribution_survey'] == channel)]
    spe1.loc[m,'count'] = len(subattri)
    spe1.loc[m,'Average_CAC_Sur'] = spe1.loc[m,'spend_AED']/spe1.loc[m,'count']

<ipython-input-201-556bd7a9e06a>:6: RuntimeWarning: divide by zero encountered in true_divide
  spe1.loc[m,'Average_CAC_Sur'] = spe1.loc[m,'spend_AED']/spe1.loc[m,'count']
<ipython-input-201-556bd7a9e06a>:6: RuntimeWarning: divide by zero encountered in true_divide
  spe1.loc[m,'Average_CAC_Sur'] = spe1.loc[m,'spend_AED']/spe1.loc[m,'count']
<ipython-input-201-556bd7a9e06a>:6: RuntimeWarning: divide by zero encountered in true_divide
  spe1.loc[m,'Average_CAC_Sur'] = spe1.loc[m,'spend_AED']/spe1.loc[m,'count']
<ipython-input-201-556bd7a9e06a>:6: RuntimeWarning: divide by zero encountered in true_divide
  spe1.loc[m,'Average_CAC_Sur'] = spe1.loc[m,'spend_AED']/spe1.loc[m,'count']
<ipython-input-201-556bd7a9e06a>:6: RuntimeWarning: divide by zero encountered in true_divide
  spe1.loc[m,'Average_CAC_Sur'] = spe1.loc[m,'spend_AED']/spe1.loc[m,'count']
<ipython-input-201-556bd7a9e06a>:6: RuntimeWarning: divide by zero encountered in true_divide
  spe1.loc[m,'Average_CAC_Sur'] = spe1.loc[m,'

In [202]:
speMs = spe1.sort_values(['channel','spend_AED'],ascending=True)
speMs.reset_index(inplace=True)
speMs.drop('index',axis='columns', inplace=True)
speMs = speMs.loc[16:39,]
speMs.reset_index(inplace=True)
speMs.drop('index',axis='columns', inplace=True)

In [203]:
speMs

,channel,date,spend_AED,count,Average_CAC_Sur
0,facebook,201907,9000,2754.0,3.267974
1,facebook,201909,10500,6939.0,1.513186
2,facebook,202001,11000,6451.0,1.705162
3,facebook,201908,13000,7508.0,1.731486
4,facebook,202002,14000,7263.0,1.927578
5,facebook,201912,16000,8568.0,1.867414
6,facebook,201910,17000,8620.0,1.972158
7,facebook,201911,23000,10816.0,2.126479
8,search,201907,13000,282.0,46.099291
9,search,201908,18500,544.0,34.007353


In [204]:
for n in range(1,len(speMs)):
    if speMs.loc[n,'channel'] == speMs.loc[(n-1),'channel']:
        speMs.loc[n,'Marginal_CAC_Sur'] = (speMs.loc[n,'spend_AED'] - speMs.loc[(n-1),'spend_AED'])/(speMs.loc[n,'count'] - speMs.loc[(n-1),'count'])
    else:
        continue
for m in [0,8,16]:
    speMs.loc[m,'Marginal_CAC_Sur'] = speMs.loc[m,'spend_AED'] /speMs.loc[m,'count']

In [207]:
speMs.to_csv("Attri_Survey.csv")

In [206]:
speMt

,channel,date,spend_AED,count,Average_CAC_Tech,Marginal_CAC_Tech
0,bing,201912,300,13.0,23.076923,23.076923
1,bing,201907,400,24.0,16.666667,9.090909
2,bing,201908,900,30.0,30.000000,83.333333
3,bing,201909,1000,51.0,19.607843,4.761905
4,bing,201910,1100,53.0,20.754717,50.000000
5,bing,201911,1300,59.0,22.033898,33.333333
6,bing,202001,2100,93.0,22.580645,23.529412
7,bing,202002,3700,179.0,20.670391,18.604651
8,display,201907,12,7.0,1.714286,1.714286
9,display,201909,13,8.0,1.625000,1.000000
